In [1]:
from Prompts import prompter
from dotenv import load_dotenv
import os
from google import genai


load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

client = genai.Client()

In [2]:
import json

# Path to your JSON file
file_path = "Datasets/network_analysis_2/na2.json"

# Load the JSON content
with open(file_path, "r") as f:
    data = json.load(f)

# Extract parts from the JSON
scenario = data["scenario"]
tools = data["tools"]
questions = data["questions"]
file=data["files"]



In [3]:
Answer_list=[]
for i in range(0, len(questions)):

    j = 0
    max_attempts = 4
    prompt = prompter.prompt_zero_shot(i, scenario, tools, file, questions)
    
    response = client.models.generate_content(   # Generate the first command
        model="gemini-2.5-flash", contents=prompt
    )
    print(response.text)

    history = [
        {"role": "system", "text": [prompt]},
        {"role": "model", "text": [f"Command generated by the model:\n{response.text}"]}
    ]

    while j < max_attempts:
        user_command_output = input("Please execute the command above and provide the output here:\n")
        user_command_output = user_command_output[:1000]  # Limit the output to 1000 characters
        history.append({"role": "user", "text": f"Output after executing the previous command:\n{user_command_output}"})
        print(f"📜{history}")

        response = client.models.generate_content(   # Generate the next command based on the history or return result
            model="gemini-2.5-flash",
            contents="\n".join([str(h) for h in history])
        )
        

        if "final answer:" in response.text.lower():
            history.append({"role": "model", "text": [f"Final answer from the model:\n{response.text}"]})
            
            text = response.text.strip()
            answer_part = text.lower().split("final answer:")[-1].strip()
            print(f"✅ Answer to the question {i+1}: {answer_part}")
            Answer_list.append(answer_part)
            break
        else:
            history.append({"role": "model", "text": [f"Command generated by the model:\n{response.text}"]})
            print(print(f"🛠️{response.text}"))
        
        j += 1
    if j == max_attempts:
        Answer_list.append("Answer not found")    

command: tshark -r Datasets/network_analysis_2/BTLOPortScan.pcap -Y "tcp.flags.syn == 1 && tcp.flags.ack == 0" -T fields -e ip.src | sort | uniq -c | sort -nr
📜[{'role': 'system', 'text': ["\nYou are tasked with solving a Blue Team Labs Online (BTLO)-style cybersecurity investigation challenge using Ubuntu terminal commands.\n\nScenario:\nThe SOC received an alert in their SIEM for ‘Local to Local Port Scanning’ where an internal private IP began scanning another internal system. Can you investigate and determine if this activity is malicious or not? You have been provided a PCAP.\n\nAvailable Tools:\ntshark, tcpdump\n\nTarget File(s):\nDatasets/network_analysis_2/BTLOPortScan.pcap\n\nQuestion:\nWhat is the IP responsible for conducting the port scan activity? (Format: X.X.X.X)\n\nInstructions:\nYou are assisting in a live investigation and must analyze the provided file(s) to answer the question using the tools listed.\n\nThere are two types of responses you can give, based on the cur

KeyboardInterrupt: 

In [4]:
print(Answer_list)

['10.251.96.4', '1-54284', 'tcp connect scan', 'Answer not found', 'upload.php', 'dbfunctions.php', 'Answer not found', 'Answer not found']


In [5]:
#saves all the answers to a file for evaluation
base_name = os.path.splitext(os.path.basename(file_path))[0]

output_file = os.path.join("Evaluation","manual","zero_shot", f"{base_name}.txt")

with open(output_file, 'w') as f:
    for answer in Answer_list:
        f.write(answer + "\n")

print(f"Answers saved to {output_file}")

Answers saved to Evaluation/manual/zero_shot/na2.txt
